# Notebook for testing the model on the test set

In [ ]:
'''
Feature extraction for the cancer dataset.
Feature 'smoke':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'drink':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'background_father':
AUSTRIA: 0
BRASIL: 1
BRAZIL: 2
CZECH: 3
GERMANY: 4
ISRAEL: 5
ITALY: 6
NETHERLANDS: 7
POLAND: 8
POMERANIA: 9
PORTUGAL: 10
SPAIN: 11
Unknown: -1
------------------------------
Feature 'background_mother':
BRAZIL: 0
FRANCE: 1
GERMANY: 2
ITALY: 3
NETHERLANDS: 4
NORWAY: 5
POLAND: 6
POMERANIA: 7
PORTUGAL: 8
SPAIN: 9
Unknown: -1
------------------------------
Feature 'pesticide':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'gender':
FEMALE: 0
MALE: 1
Unknown: -1
------------------------------
Feature 'skin_cancer_history':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'cancer_history':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'has_piped_water':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'has_sewage_system':
False: 0
True: 1
Unknown: -1
------------------------------
Feature 'grew':
FALSE: 0
TRUE: 1
Unknown: -1
------------------------------
Feature 'changed':
FALSE: 0
TRUE: 1
Unknown: -1
------------------------------
'''

In [4]:
# randomly generate three numeric features base on the original features
import random

def random_value(mapping):
    """Randomly selects a value from the provided mapping dictionary."""
    return random.choice(list(mapping.values()))

def generate_random_features():
    features = {
        'smoke': {False: 0, True: 1, 'Unknown': -1},
        'drink': {False: 0, True: 1, 'Unknown': -1},
        'background_father': {'AUSTRIA': 0, 'BRASIL': 1, 'BRAZIL': 2, 'CZECH': 3, 'GERMANY': 4, 'ISRAEL': 5, 'ITALY': 6, 'NETHERLANDS': 7, 'POLAND': 8, 'POMERANIA': 9, 'PORTUGAL': 10, 'SPAIN': 11, 'Unknown': -1},
        'background_mother': {'BRAZIL': 0, 'FRANCE': 1, 'GERMANY': 2, 'ITALY': 3, 'NETHERLANDS': 4, 'NORWAY': 5, 'POLAND': 6, 'POMERANIA': 7, 'PORTUGAL': 8, 'SPAIN': 9, 'Unknown': -1},
        'pesticide': {False: 0, True: 1, 'Unknown': -1},
        'gender': {'FEMALE': 0, 'MALE': 1, 'Unknown': -1},
        'skin_cancer_history': {False: 0, True: 1, 'Unknown': -1},
        'cancer_history': {False: 0, True: 1, 'Unknown': -1},
        'has_piped_water': {False: 0, True: 1, 'Unknown': -1},
        'has_sewage_system': {False: 0, True: 1, 'Unknown': -1},
        'grew': {False: 0, True: 1, 'Unknown': -1},
        'changed': {False: 0, True: 1, 'Unknown': -1}
    }

    # Generate a random value for each feature
    random_feature_values = {feature: random_value(mapping) for feature, mapping in features.items()}
    return random_feature_values

random_features = generate_random_features()

In [4]:
# try if the image processor is working
import tensorflow as tf

# image processing for model
def img_preprocessing(img):
    """ Image preprocessing function """
    img = tf.io.read_file(img)  # Read the image file
    img = tf.image.decode_png(img, channels=3)  # Decode the PNG image
    img = tf.image.resize(img, (150, 150))  # Resize the image, image size is (256, 256)
    img = tf.cast(img, tf.float32) / 255.0  # Normalize pixel values to [0, 1] range
    return img

# image path
img_path = 'testing_data/test3.png'

img = img_preprocessing(img_path)

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [1]:
# test if the model is working
from keras.models import load_model

# Load the model
model = load_model('pretrained_image_model.h5')
model.summary()

2024-05-07 05:35:10.666319: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-07 05:35:10.666364: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-05-07 05:35:16.993923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-07 05:35:16.993961: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-07 05:35:16.993987: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-ab1841): /proc/driver/nvidia/version does not exist
2024-05-07 05:35:17.012521: I tensorflow/core/platform/cpu_

Model: "model_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_42 (InputLayer)       [(None, 150, 150, 3)]     0         
                                                                 
 xception (Functional)       (None, 5, 5, 2048)        20861480  
                                                                 
 global_average_pooling2d_20  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_54 (Dense)            (None, 25)                51225     
                                                                 
 dropout_20 (Dropout)        (None, 25)                0         
                                                                 
 dense_55 (Dense)            (None, 7)                 182       
                                                          

In [6]:
# try to predict the image
def predict_image(model, img):
    """ Predict the image """
    img = tf.expand_dims(img, axis=0)  # Expand the image dimensions
    prediction = model.predict(img)  # Predict the image
    return prediction

prediction = predict_image(model, img)
prediction

array([[-2.1256373 , -1.1598643 ,  0.9277239 , -2.0813558 ,  0.34929943,
         2.9606695 , -1.1918007 ]], dtype=float32)